# Senior Top 10

Created by Michael George (AKA Logiqx)

Note: This can only be run after "Overs 40s" and "Percentile Ranks"

Link: https://www.speedsolving.com/forum/showthread.php?54128-How-fast-are-the-over-40-s-in-competitions

In [33]:
from EventsLib import *

## Read Event Results from CSV

Read event data from CSV into memory, prior to processing

In [34]:
import os, csv

class EventResults:
    
    def __init__(self):
        """Initialisise the event results"""
        
        self.event = None
        self.results = []
        self.total = 0


    def readResults(self, basename, event):
        """Read event results from CSV into memory"""
        
        self.event = event
        self.results = []
        self.total = 0

        # Read rows using the CSV reader
        fn = os.path.join('data', 'public', basename, self.event[0] + '.csv')
        with open(fn, 'rb') as f:
            csvReader = csv.reader(f)
            
            # Process each row individually
            for inputRow in csvReader:
                
                # Pack out results with zeros
                while (int(inputRow[0]) > len(self.results)):
                    self.results.append([0, self.total])
                
                count = int(inputRow[1])
                self.total += count
                self.results.append([count, self.total])

## Read Partial Results from CSV

Read event data from CSV into memory, prior to processing

In [35]:
import os, csv, re

class PartialResults:
    
    def __init__(self):
        """Initialisise the partial results"""
        
        self.event = None
        self.persons = {}
        self.results = {}


    def readPersons(self, basename):
        """Read seniors from CSV into memory"""
        
        self.persons = {}

        # Read rows using the CSV reader
        fn = os.path.join('data', 'public', basename + '.csv')
        with open(fn, 'rb') as f:
            csvReader = csv.reader(f)
            
            # Process each row individually
            for person in csvReader:
                self.persons[person[0]] = person[1:]
        
                
    def readResults(self, basename, event):
        """Read event results from CSV into memory"""
        
        self.event = event
        self.results = {}

        # Read rows using the CSV reader
        fn = os.path.join('data', 'public', basename, self.event[0] + '.csv')
        with open(fn, 'rb') as f:
            csvReader = csv.reader(f)
            
            # Process each row individually
            for inputRow in csvReader:
                
                result = int(inputRow[1]) / 100
                
                if result > self.event[4]:
                    result = self.event[4]
                
                if self.results.has_key(result):
                    self.results[result] += [inputRow]
                else:
                    self.results[result] = [inputRow]

## Analyse Results

Process all three sets of results simultaneously

In [36]:
class EventAnalysis:
    
    def __init__(self):
        """Initialisise the event analysis"""
        
        self.event = None
        self.seniorResults = EventResults()
        self.partialResults = PartialResults()


    def readPersons(self):
        """Read seniors from CSV into memory"""

        self.partialResults.readPersons('seniors')


    def readResults(self, event):
        """Read event results from CSV into memory"""

        # Skip processing if no cutoff is defined
        if event[4] > 0:
            self.event = event
            self.seniorResults.readResults('all_senior_averages', self.event)
            self.partialResults.readResults('partial_senior_averages', self.event)
        else:
            self.event = None


    def getHtml(self):
        """Get the HTML for the event"""
        
        html = ''
        
        if self.event:
            html += '<details>'
            html += '<summary>%s</summary>\n' % self.event[1]
            html += '<table>\n'
            html += '<tr>'
            for field in ['<img width="80" height="1"><br/>Rank', '<img width="400" height="1"><br/>Person(s)', '<img width="80" height="1"><br/>Result(s)']:
                html += '<td><b>%s</b></td>' % field
            html += '</tr>\n'

            i = 0
            count = 0
            pos = 1
            while i < len(self.seniorResults.results):
                
                seniorResult = self.seniorResults.results[i]
                
                if self.partialResults.results.has_key(i):
                    knownResults = self.partialResults.results[i]
                else:
                    knownResults = []

                known = len(knownResults)
                unknown = seniorResult[0] - len(knownResults)

                # The result may be a time or count
                if self.event[3] == 'time':
                    result = formatTime(i)
                else:
                    result = str(i)
                    
                if (i >= self.event[4]):
                    result = 'sup-' + result
                else:
                    result += '.xx'

                if unknown == 0:
                    for knownResult in knownResults:

                        link = '<a href="https://www.worldcubeassociation.org/results/p.php?i=%s#%s">%s</a>' % \
                                (knownResult[0], self.event[0], knownResult[0]);
                        person = self.partialResults.persons[knownResult[0]]
                        name = re.sub(' \(.*\)', '', person[0])
                        persons = '%s, %s - %s' % (name, person[1], link)
                        
                        knownValue = int(knownResult[1])
                        if self.event[3] == 'time':
                            results = formatTime(knownValue / 100, knownValue % 100)
                        else:
                            results = str(knownValue)

                        html += '<tr>'
                        html += '<td>%s</td>' % pos
                        html += '<td>%s</td>' % persons
                        html += '<td>%s</td>' % results
                        html += '</tr>\n'

                        pos += 1

                else:
                    persons = ''
                    results = ''
                    if len(knownResults) > 0:
                        for knownResult in knownResults:
                            if persons:
                                persons += '<br/>'
                                results += '<br/>'
                                
                            link = '<a href="https://www.worldcubeassociation.org/results/p.php?i=%s#%s">%s</a>' % \
                                    (knownResult[0], self.event[0], knownResult[0]);
                            person = self.partialResults.persons[knownResult[0]]
                            name = re.sub(' \(.*\)', '', person[0])
                            persons += '%s, %s - %s' % (name, person[1], link)

                            knownValue = int(knownResult[1])
                            if self.event[3] == 'time':
                                results += formatTime(knownValue / 100, knownValue % 100)
                            else:
                                results += str(knownValue)
                            
                        if unknown > 0:
                            persons += '<br/>+ %d unknown%s' % (unknown, 's' if unknown > 1 else '')
                            results += '<br/>%s' % result
                    else:
                        if unknown > 1:
                            persons = '%d unknown%s' % (unknown, 's' if unknown > 1 else '')
                        elif unknown == 1:
                            persons = 'Unknown'
                            
                        results = '%s' % result

                    html += '<tr>'
                    html += '<td>%s</td>' % (pos if seniorResult[0] == 1 else '%d-%d' % (pos, seniorResult[1]))
                    html += '<td>%s</td>' % persons
                    html += '<td>%s</td>' % results
                    html += '</tr>\n'

                    pos += seniorResult[0]

                    count = seniorResult[1]
                    
                i += 1

            html += '</table>\n'
            html += '</details>\n\n'
        
        return html

## Analyse Events

Process the events one-by-one

In [37]:
html = '''<h1>Background</h1>
<p>This project started on the <a href="https://www.speedsolving.com/forum/threads/how-fast-are-the-over-40s-in-competitions.54128/">SpeedSolving.com</a> forum and was last updated %s.</p>
<p>The statistics below have been calculated using <a href="sql/senior_aggregates.sql">aggregated data</a> from the WCA database.</p>
\n''' % '2019-02-01'

html += '<h1>%s</h1>\n\n' % 'Official Averages'

eventAnalysis = EventAnalysis()
eventAnalysis.readPersons()

for event in events:
    eventAnalysis.readResults(event)
    html += eventAnalysis.getHtml()
    
with open("Senior Rankings.md", 'w') as f:
    f.write(html)